# Sampling Si GB phases

In [15]:
import sys
sys.path.append('../interfacemaster/')

In [16]:
from cellcalc import get_primitive_hkl
from interface_generator import core, print_near_axis, \
convert_vector_index, write_trans_file, write_LAMMPS, RBT_deletion_one_by_one
from numpy import array, dot, round, var, average, ceil
from numpy.linalg import inv, norm, det

In [17]:
my_interface = core('cif_files/Si_mp-149_conventional_standard.cif',\
                    'cif_files/Si_mp-149_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


# Remember to normalize the size of the cell according to your computed lattice parameter!

In [18]:
factor = 5.43356/5.468728

In [19]:
my_interface.conv_lattice_1 = my_interface.conv_lattice_1 * factor
my_interface.conv_lattice_2 = my_interface.conv_lattice_2 * factor

In [20]:
my_interface.lattice_1 = my_interface.lattice_1 * factor

In [21]:
my_interface.lattice_2 = my_interface.lattice_2 * factor

In [22]:
axis = convert_vector_index(my_interface.conv_lattice_1, my_interface.lattice_1, [1,0,0])
axis

array([-1., -1.,  1.])

In [23]:
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)
#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(axis,36.0,1,0.01)

[5.4335600e+00 0.0000000e+00 4.4408921e-16]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  1]
 [ 1  1 -2]
 [-1  1  1]]; sigma_1 = 5

U2 = 
[[ 1  1  1]
 [ 1  0 -2]
 [-1  1  0]]; sigma_2 = 5

D = 
[[ 1.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  1.00e+00  1.79e-06]
 [ 0.00e+00 -1.79e-06  1.00e+00]]

axis = [5.4335600e+00 0.0000000e+00 4.4408921e-16] ; theta = 36.87



In [24]:
hkl = get_primitive_hkl(array([0,1,2]), my_interface.conv_lattice_1, my_interface.lattice_1)
hkl

array([-1, -2, -3])

# Generate a bicrystal, check the distance from diffrent terminating atomic planes to the boundary!

In [26]:
my_interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)
my_interface.get_bicrystal(xyz_1 = [9,1,1], xyz_2 = [9,1,1],dp1 = 0, dp2 = 0,\
                           filetype = 'LAMMPS', filename = 'atominfile', vx = 10)

cell 1:
[[-1 -1  3]
 [ 1 -1 -3]
 [ 3  1  1]]
cell 2:
[[-1 -1  3]
 [ 1 -1 -3]
 [ 3  1  1]]


In [27]:
print(my_interface.dp_list_1[0], my_interface.dp_list_2[0])

0.6074856784138376 0.0


This means if dp1 < 0.607..., left GB have first termination; if dp1 > 0.607..., left GB have second termination. For the right GB, if dp2 = 0.0, right GB has first termination...

# According to the length of CNID vectors, determining RBT searching grids

In [28]:
grid_1 = int(ceil(norm(my_interface.CNID[:,0])/0.2))
grid_2 = int(ceil(norm(my_interface.CNID[:,1])/0.2))
print(grid_1, grid_2)

28 34


# Adjust the orientation of CNID and express it by fractional coordinates 
We have adjusted the orientation of the bicrystal to make it look better in the VESTA or Ovito, now we need to do this for CNID as well. Also, we need to express CNID in fractional coordinates as our atoms were all in fraction coordinates before being written to the LAMMPS input file, we should adjust their positions by adjusting fractional coordinates of them.

In [29]:
CNID = dot(my_interface.orient, my_interface.CNID)
CNID = dot(inv(my_interface.lattice_bi), CNID)
CNID

array([[-9.09315636e-35, -2.71099180e-19],
       [-1.00000000e+00,  5.00000000e-01],
       [-5.29903877e-50, -5.00000000e-01]])

# Make a folder to save file (might be very large)

In [31]:
import os
os.mkdir('sigma5_st_AB')
os.chdir('sigma5_st_AB')

# The nearest atomic distance in the lattice

In [32]:
d_nearest = 2.35820

# Set the GB dimensions. You need different dimensions to sampling the density of atoms in a complete atomic plane!


This is what I used, which has too many files for tutorial \
testing but you can see how I make a fine sampling

size_list = [[1, 2], [1, 3], [1, 5], [1, 7], [1, 8],\
             [2, 2], [2, 3], [2, 5], [2, 7], [2, 8],\
             [3, 2], [3, 3], [3, 5], [3, 7], [3, 8],\
             [4, 7], [5, 5], [7, 7]]


# Defining the GB position (you only translate the right crystal) 109


# Defining the region for atom deletion (several atomic planes is enough) xlo 106; xhi 111.5

In [33]:
size_list = [[1, 1], [1, 2]]

for i in size_list:
    name = '{0}{1}'.format(i[0],i[1])
    os.mkdir(name)
    os.chdir(name)
    my_interface.get_bicrystal(xyz_1 = [9,i[0],i[1]], xyz_2 = [9,i[0],i[1]],dp1 = 0, dp2 = 0,\
                           filetype = 'LAMMPS', filename = 'atominfile', vx = 10)
    RBT_deletion_one_by_one(my_interface.lattice_bi, my_interface.atoms_bi, my_interface.elements_bi, CNID, \
                       [grid_1,grid_2], 109, d_nearest, xlo = 106, xhi = 111.5)
    os.chdir(os.path.pardir)

# Note that we will use a single LAMMPS input to run simulation for all these files. For this purpose, you need some files saving variables for the LAMMPS to read your generated files. We have three loops:
1. Looping the GB expansion
2. Looping the RBT
3. Looping deletion of atoms

In [38]:
file_epl = open('expansion_loop','w')
file_cl = open('cnid_loop','w')
file_dl = open('deleting_loop','w')

for i in size_list:
    expansion_name = '{0}{1}'.format(i[0],i[1])
    os.chdir(expansion_name)
    for j in range(grid_1):
        for k in range(grid_2):
            cnid_name = '{0}_{1}'.format(j,k)
            os.chdir(cnid_name)
            for h in os.listdir():
                file_epl.write('{0}{1}\n'.format(i[0],i[1]))
                file_dl.write('{}\n'.format(h)) 
                file_cl.write('{} \n'.format(cnid_name))
            os.chdir(os.path.pardir)
    os.chdir(os.path.pardir)
file_epl.close()
file_cl.close()
file_dl.close()

# Now, we define the 'bulk' and 'middle' regions by making a 'blockfile' in the cell to compute GB properties in LAMMPS. 
middle: x_range (69, 149)

bulk: x_range (39, 69)

In [39]:
my_interface.define_lammps_regions(['middle','bulk'],[69,39],[149,69])

# Now, copy the 'Si.tersoff' and 'proto.in' in the 'test_file' folder to this folder, then run LAMMPS!